In [4]:
import numpy as np
import matplotlib.pyplot as plt
from data_load import load_data, pmumu, nll, lam_i

data, flux = load_data()
dm2_approx = 0.002512121212121212
theta_approx = 0.677214566183642

For the choices of approximation, the values of dm2 using a fine range and then the values of theta, over a fine range, gathered from using the fine range dm2 value are used (dm2 --> dm2 fine (used) --> use dm2 value for theta --> theta fine (used)) just for consistency.

Just for clarity, the value of $\lambda_i$ is the expected counts in each energy bin:

$$
\lambda_i(\theta, \Delta m^2) = \Phi_i^{Unoscillated}P_{\mu\mu}(\theta_i, \Delta m^2)
$$

For parabolic minimsation we are using:

$$
f(x) = NLL(\theta_{fixed}, x), \text{where } x \equiv \Delta m^2
$$

And similarly for $\theta$ but keeping $\Delta m^2$ fixed, this is essentially a cost function.

We need three points say $x_0, x_1, x_2$ and then to evaluate $f(x_0) = y_0, f(x_1) = y_1, \text{and } f(x_2) = y_2$ (as from the notes)

We need to fit a quadratic (Lagrange Interpolating Polynomial) through these points :

$$
P_2(x) = \frac{(x-x_1)(x-x_2)}{(x_0-x_1)(x_0-x_2)}y_0 + \frac{(x-x_0)(x-x_2)}{(x_1-x_0)(x_1-x_2)}y_1 + \frac{(x-x_0)(x-x_1)}{(x_2-x_0)(x_2-x_1)}y_2
$$

We will then differentiate $P_2(x)$, setting the derivative to 0 and we can solve for x and rearrange to get:

$$
x_3 =\frac{1}{2} \frac{(x_2^2-x_1^2)y_0 + (x_0^2-x_2^2) y_1 + (x_1^2-x_0^2)y_2}{(x_2-x_1)y_0 +(x_0-x_2)y_1 + (x_1 - x_0)y_2}
$$

$x_3$ is our new estimate of the minimum, we then evaluate $y_3 = f(x_3)$ and loop, keeping the best three of $(x_0,y_0), (x_1, y_1) \text{ and } (x_2,y_2)$, repeat until $|x_3-x_1|$ < than a set tolerance, obviously replacing x with whatever value we're interested in ($\Delta m^2 \text{ or } \theta$)

For dm2 we got values ranging from $2.5-2.53 \times 10^{-3} eV^2$ and I will attempt to use a convergence criteria of $1\times 10^{-5}$ but if this takes too long then I may switch to $1\times 10^{-4}$ 

For theta we get a bit more change with values from 0.66 to 0.69 thus for now I will set the convergence criteria to $1\times 10^{-3}$

In [ ]:
def parabolic(f, x0, x1, x2, tol, max_iter=300):
    """
    1D parabolic minimiser 

    f = NLL function for 1 var
    x0,x1,x2 = initial points
    tol = convergence critera

    Returns minimum of x and minimum of NLL at x_min
    """
    f0 = f(x0)
    f1 = f(x1)
    f2 = f(x2)

    for i in range(max_iter):
        x3 = 0.5*((x2**2 - x1**2) * f0 + (x0**2 - x2**2) * f1 + (x1**2 - x0**2) * f2)/((x2 - x1) * f0 + (x0 - x2) * f1 + (x1 - x0) * f2)
        f3 = f(x3)

        # check convergence criteria and then input values 
        if abs(x3 - x1) < tol:
            x_mins = np.array([x0, x1, x2, x3])
            f_mins = np.array([f0, f1, f2, f3])
            min_val = np.argmin(f_mins)
            return x_mins[min_val], f_mins[min_val]

        # keep best 3 points with most minimised f
        x_mins = np.array([x0, x1, x2, x3])
        f_mins = np.array([f0, f1, f2, f3])

        # Sort by x so the bracket stays ordered
        sort = np.argsort(x_mins)
        x_mins = x_mins[sort]
        f_mins = f_mins[sort]

        # replace initial guesses with newly minimised approximations
        x0, x1, x2 = x_mins[0], x_mins[1], x_mins[2]
        f0, f1, f2 = f_mins[0], f_mins[1], f_mins[2]

    # if not converged at max iterations then just return best value
    x_mins = np.array([x0, x1, x2])
    f_mins = np.array([f0, f1, f2])
    min_val = np.argmin(f_mins)
    return x_mins[min_val], f_mins[min_val]

# Test Function

In [ ]:
def test(x):
    return (x-4)**2+5

min_test, f_min_test = parabolic(test, 0, 6, 10, tol = 1e-3)
print(min_test, f_min_test)

4.0 5.0


In [30]:
def nll_dm2(dm2):
    return nll(theta_approx, dm2)

def nll_theta(theta):
    return nll(theta, dm2_approx)

In [31]:
# choosing equally spaced starting points for dm2 initial guesses so that they bracket the min

dm2_0 = 1.5e-3
dm2_1 = 2.4e-3
dm2_2 = 3.5e-3

t0 = 0.4
t1 = 0.7
t2 = 1.0

dm2_min, nll_dm2_min = parabolic(nll_dm2, dm2_0, dm2_1, dm2_2, tol = 1e-5)


theta_min, nll_theta_min = parabolic(nll_theta, t0, t1, t2, tol = 1e-3)

In [32]:
print(dm2_min, theta_min)

0.0024473468435815774 0.6751499971144296
